In [406]:
model_path = "models/drumloop2/drumloop2.keras"
maxlen = 8
prompt = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
sequence_length = 2

In [407]:
from keras.models import load_model
import numpy as np
from keras.utils import to_categorical
global model
model = load_model(model_path)
model.summary()

Model: "drumloop2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 8, 4)]            0         
                                                                 
 gru_16 (GRU)                (None, 24)                2160      
                                                                 
 dense_15 (Dense)            (None, 4)                 100       
                                                                 
Total params: 2,260
Trainable params: 2,260
Non-trainable params: 0
_________________________________________________________________


In [408]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = preds[0]
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # print(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [432]:
def generate(prompt=[], length=4, temperature=1.0, include_prompt=False, verbose=0):
    # generate a sequence of given length       
    if len(prompt) == 0:
        # if it's there's no prompt, use an array of zeroes
        prompt = np.zeros((maxlen), dtype="uint8")
        print(f"No prompt provided, using {maxlen} zeroes instead.")
    elif len(prompt) < maxlen:
        # if prompt is too short, pad it with zeroes from the left to match correct input shape
        prompt = np.pad(prompt, (maxlen - len(prompt), 0), 'constant', constant_values=(0, 0))
        print("Prompt too short, padded to length: ", maxlen)
    elif len(prompt) > maxlen:
        # if it's too long, then trim it
        prompt = prompt[-maxlen:]
        print(f"Prompt too long, using {maxlen} last elements: ")
    prompt_ = to_categorical(prompt, num_classes=4)
    prompt_ = np.array([prompt_])
    
    seq = []
    for i in range(length):
        # make prediction based on prompt
        ps = model.predict(prompt_, verbose=verbose)
        # sample from predictions
        p_label = sample(ps, temperature)
        # add sampled label to sequence
        seq.extend([int(p_label)])
        # one-hot encode sampled label
        p_label_ = to_categorical(p_label, num_classes=4)
        p_label_ = np.array([[p_label_]])
        # append encoded label to the prompt for next prediction
        prompt_ = np.append(prompt_, p_label_, axis=1)
    if include_prompt:
        prompt = np.append(prompt, seq)
        return prompt
    else:
        return seq

In [433]:
# seq = generate(prompt=prompt, length=sequence_length)
# seq

In [467]:
def gen_thread():
    global prompt
    while True:
        print("Generating sequence...")
        seq = generate(prompt=prompt, length=sequence_length)
        prompt.extend(seq) # add generated sequence to the prompt
        prompt = prompt[-maxlen:] # take only last maxlen elements
        print(prompt)